In [5]:
import re

def string2ngrams(n,string):
    #lst = re.split("[\s]",string)
    lst = re.split("[^0-9a-z]",string)
    ngrams = zip(*[lst[i:] for i in xrange(n)])
    return ngrams

def countNGrams(n,sentences):
    ngrams = sentences.map(lambda string:string2ngrams(n,string))
    counter = ngrams.flatMap(lambda ngram:(" ".join(ngram),1)).reduceByKey(lambda a,b:a+b)
    return counter

def preprocessFile2Sentences(filename):
    textRDD = sc.newAPIHadoopFile(filename,
                              'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
                              'org.apache.hadoop.io.LongWritable',
                              'org.apache.hadoop.io.Text',
                               conf={'textinputformat.record.delimiter': "\r\n\r\n"}) \
            .map(lambda x: x[1])

    sentences=textRDD.flatMap(lambda x: x.split(". "))
    sentences=sentences.map(lambda sent:" ".join(re.findall('\w+',sent.lower())))
    return sentences

def get_Freq_ngramRDD(n, sentences):
    ngrams = sentences.map(lambda string:string2ngrams(n,string))
    collection_of_ngrams = ngrams.flatMap(lambda x:(map(lambda y:(y,1),x)))
    sorted_counter = collection_of_ngrams.reduceByKey(lambda a,b:a+b).map(lambda (x,y):(y,x)).sortBy(lambda x:x[0] ,False)
    return sorted_counter

def printOutput(n,freq_ngramRDD):
    top=freq_ngramRDD.take(5)
    print '\n============ %d most frequent %d-grams'%(5,n)
    print '\nindex\tcount\tngram'
    for i in range(5):
        #print top[i][1]
        print '%d.\t%d: \t"%s"'%(i+1,top[i][0],' '.join(top[i][1]))


#filename = '../Data/mydic.txt'
filename = '../Data/Moby-Dick.txt'
sentences = preprocessFile2Sentences(filename)
#so far, each element of sentences is a well preprocessed sentence

for n in range(1,6):
#    Put your logic for generating the sorted n-gram RDD here and store it in freq_ngramRDD variable
    freq_ngramRDD = get_Freq_ngramRDD(n,sentences)
    printOutput(n,freq_ngramRDD)


============ 5 most frequent 1-grams

index	count	ngram
1.	14620: 	"the"
2.	6732: 	"of"
3.	6502: 	"and"
4.	4799: 	"a"
5.	4706: 	"to"

============ 5 most frequent 2-grams

index	count	ngram
1.	1906: 	"of the"
2.	1193: 	"in the"
3.	746: 	"to the"
4.	444: 	"from the"
5.	413: 	"the whale"

============ 5 most frequent 3-grams

index	count	ngram
1.	116: 	"the sperm whale"
2.	109: 	"of the whale"
3.	88: 	"the white whale"
4.	64: 	"one of the"
5.	60: 	"of the sea"

============ 5 most frequent 4-grams

index	count	ngram
1.	43: 	"of the sperm whale"
2.	27: 	"the sperm whale s"
3.	20: 	"at the same time"
4.	18: 	"project gutenberg tm electronic"
5.	18: 	"of the whale s"

============ 5 most frequent 5-grams

index	count	ngram
1.	13: 	"the project gutenberg literary archive"
2.	13: 	"project gutenberg literary archive foundation"
3.	12: 	"project gutenberg tm electronic works"
4.	11: 	"of the sperm whale s"
5.	10: 	"and at the same time"
